### **Sabrina Zheng | Group 6 | Chapter 6 | Quesiton 5**

_Written in collaboration with ChatGPT from OpenAI to improve understanding, assist with the explanation of the query, and enhance formatting and display of the queries._

**Question 5:**

\-- 5

\-- Write a query that returns customer and employee pairs 

\-- that had order activity in both January 2016 and February 2016

\-- but not in 2015

\-- Tables involved: TSQLV4 database, Orders table

\--Desired output

custid      empid

\----------- -----------

67          1

46          5

In [2]:
--Query using TSQLV4 database
USE TSQLV4;

SELECT DISTINCT o1.custid, o1.empid
FROM Orders o1
JOIN Orders o2 ON o1.custid = o2.custid AND o1.empid = o2.empid
WHERE o1.orderdate >= '2016-01-01' AND o1.orderdate < '2016-02-01'
  AND o2.orderdate >= '2016-02-01' AND o2.orderdate < '2016-03-01'
  AND NOT EXISTS (
      SELECT 1
      FROM Orders o3
      WHERE o3.custid = o1.custid AND o3.empid = o1.empid
        AND o3.orderdate >= '2015-01-01' AND o3.orderdate < '2016-01-01'
  )
  AND NOT EXISTS (
      SELECT 1
      FROM Orders o4
      WHERE o4.custid = o1.custid AND o4.empid = o1.empid
        AND o4.orderdate < '2016-01-01'
  );


--Query using Northwinds2022TSQLV7
USE Northwinds2022TSQLV7;

SELECT DISTINCT o1.CustomerId, o1.EmployeeId
FROM Sales.[Order] o1
JOIN Sales.[Order] o2 ON o1.CustomerId = o2.CustomerId AND o1.EmployeeId = o2.EmployeeId
WHERE o1.OrderDate >= '2016-01-01' AND o1.OrderDate < '2016-02-01'
  AND o2.OrderDate >= '2016-02-01' AND o2.OrderDate < '2016-03-01'
  AND NOT EXISTS (
      SELECT 1
      FROM Sales.[Order] o3
      WHERE o3.CustomerId = o1.CustomerId AND o3.EmployeeId = o1.EmployeeId
        AND o3.OrderDate >= '2015-01-01' AND o3.OrderDate < '2016-01-01'
  )
  AND NOT EXISTS (
      SELECT 1
      FROM Sales.[Order] o4
      WHERE o4.CustomerId = o1.CustomerId AND o4.EmployeeId = o1.EmployeeId
        AND o4.OrderDate < '2016-01-01'
  );

(2 rows affected)

(2 rows affected)

Total execution time: 00:00:00.203

custid,empid
46,5
67,1


CustomerId,EmployeeId
46,5
67,1


**Explanation for TSQLV4 and Northwinds2022TSQLV7**

**Proposition:** Write a query that returns the pairs of customer and employees that ordered in January 2016 and Febuary 2016 but not in January 2015

**Tables:** 

- Orders (Sales.\[Order\])

**Columns:**

- custid (CustomerID)
- empid (EmployeeID)
- orderdate (OrderDate)

**Predicate:**

- Selects distinct pairs from custid and empid from the Orders table and self joins them on the Orders table based on the custid and empid that match
- The WHERE filters the data based on the orders that are placed by the same customer and employee in January 2016, 2016-01-01 and 2016-02-01 and February 2016, 2016-02-01 and 2016-03-01
- Then exclude the pairs that are placed by the same customer and employee between January 2015 and January 2016 and before January 2016

## **Part 2:**

**Chaper 6: Lines 88 - 127** 

\--------------------------------------------------------------------- 

\-- The EXCEPT Operator 

\---------------------------------------------------------------------

**Proposition:** Write a query using HR.Employees and Sales.Customers table to retrieve the country, region, and city using a CTE named EXCEPT\_ALL without any duplicates

**Tables:**

- <span style="color: var(--vscode-foreground);">HR.Employees (</span>HumanResources.\[Employee\])
    
- Sales.Customers (Sales.\[Customer\])
    

**Columns:**

- <span style="color: var(--vscode-foreground);">country (EmployeeCountry/CustomerCountry)</span>    
    
- region (EmployeeRegion/CustomerRegion)
    
- city (EmployeeCity/CustomerCity)
    

**Predicate:**

- The CTE is defined and named EXCEPT\_ALL and takes the rows from the HR.Employees table and removes whatever is duplicated based on the country, region, and city
- A row number is assigned to each group using the ROW\_NUMBER() function
- The PARTITION BY divides the rows based on the country, region, and city
- Then the ORDER BY clause is used to order the rows arbitrarily
- Then EXCEPT is used to remove rows from the Employees table that exist in the Sales.Customers table based on the country, region, and city
- The final SELECT statement retrieves the country, region and city from the EXCEPT\_ALL CTE

In [3]:
--Query using TSQLV4 database
USE TSQLV4;
-- The EXCEPT ALL Multiset Operator (Optional, Advanced)
WITH EXCEPT_ALL
AS
(
  SELECT
    ROW_NUMBER() 
      OVER(PARTITION BY country, region, city
           ORDER     BY (SELECT 0)) AS rownum,
    country, region, city
  FROM HR.Employees

  EXCEPT

  SELECT
    ROW_NUMBER() 
      OVER(PARTITION BY country, region, city
           ORDER     BY (SELECT 0)),
    country, region, city
  FROM Sales.Customers
)
SELECT country, region, city
FROM EXCEPT_ALL;


--Query using Northwinds2022TSQLV7
USE Northwinds2022TSQLV7;
-- The EXCEPT ALL Multiset Operator (Optional, Advanced)
WITH EXCEPT_ALL
AS
(
  SELECT
    ROW_NUMBER() 
      OVER(PARTITION BY EmployeeCountry, EmployeeRegion, EmployeeCity
           ORDER     BY (SELECT 0)) AS rownum,
    EmployeeCountry, EmployeeRegion, EmployeeCity
  FROM HumanResources.[Employee]

  EXCEPT

  SELECT
    ROW_NUMBER() 
      OVER(PARTITION BY CustomerCountry, CustomerRegion, CustomerCity
           ORDER     BY (SELECT 0)),
    CustomerCountry, CustomerRegion, CustomerCity
  FROM Sales.[Customer]
)
SELECT EmployeeCountry, EmployeeRegion, EmployeeCity
FROM EXCEPT_ALL;

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.413

country,region,city
USA,WA,Redmond
USA,WA,Tacoma
USA,WA,Seattle


EmployeeCountry,EmployeeRegion,EmployeeCity
USA,WA,Redmond
USA,WA,Tacoma
USA,WA,Seattle


\---------------------------------------------------------------------

\-- Precedence

\---------------------------------------------------------------------

**Proposition:** Write a query to retrive the combinations of country, region, and city from Production.Suppliers table that are not in HR.Employees table but coincide with the entries in Sales.Customers table using the EXCEPT and INTERSECT operator

**Tables:**

- Production.Suppliers (Production.\[Supplier\])
- HR.Employees (HumanResources.\[Employee\])
- Sales.Customers (Sales.\[Customer\])

**Columns:**

- country (SupplierCountry/EmployeeCountry/CustomerCountry)
- region (SupplierRegion/EmployeeRegion/CustomerRegion)
- city (CustomerCountry/CustomerRegion/CustomerCity)

**Predicate:**

- EXCEPT operation is performed between the country, region, and city from Production.Suppliers and the HR.Employees table
- Then INTERSECT between the results from the EXCEPT operation of the country, region, and city from the Sales.Customers table
- The parentheses is used to specify the order of operations to ensure that EXCEPT is done before the INTERSECT operation. 
- The query results in the country, region, and city from the Productions.Suppliers table that are not in HR.Employees table but intersect with the Sales.Customers table

In [5]:
--Query using TSQLV4 database
USE TSQLV4;
-- INTERSECT Precedes EXCEPT
SELECT country, region, city FROM Production.Suppliers
EXCEPT
SELECT country, region, city FROM HR.Employees
INTERSECT
SELECT country, region, city FROM Sales.Customers;

-- Using Parenthesis
(SELECT country, region, city FROM Production.Suppliers
 EXCEPT
 SELECT country, region, city FROM HR.Employees)
INTERSECT
SELECT country, region, city FROM Sales.Customers;


--Query using Northwinds2022TSQLV7
USE Northwinds2022TSQLV7;
-- INTERSECT Precedes EXCEPT
SELECT SupplierCountry, SupplierRegion, SupplierCity FROM Production.[Supplier]
EXCEPT
SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.[Employee]
INTERSECT
SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.[Customer];

-- Using Parenthesis
(SELECT SupplierCountry, SupplierRegion, SupplierCity FROM Production.[Supplier]
 EXCEPT
 SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.[Employee])
INTERSECT
SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.[Customer];

(28 rows affected)

(3 rows affected)

(28 rows affected)

(3 rows affected)

Total execution time: 00:00:00.130

country,region,city
Australia,NSW,Sydney
Australia,Victoria,Melbourne
Brazil,NULL,Sao Paulo
Canada,Québec,Montréal
Canada,Québec,Ste-Hyacinthe
Denmark,NULL,Lyngby
Finland,NULL,Lappeenranta
France,NULL,Annecy
France,NULL,Montceau
France,NULL,Paris


country,region,city
Canada,Québec,Montréal
France,NULL,Paris
Germany,NULL,Berlin


SupplierCountry,SupplierRegion,SupplierCity
Australia,NSW,Sydney
Australia,Victoria,Melbourne
Brazil,NULL,Sao Paulo
Canada,Québec,Montréal
Canada,Québec,Ste-Hyacinthe
Denmark,NULL,Lyngby
Finland,NULL,Lappeenranta
France,NULL,Annecy
France,NULL,Montceau
France,NULL,Paris


SupplierCountry,SupplierRegion,SupplierCity
Canada,Québec,Montréal
France,NULL,Paris
Germany,NULL,Berlin
